In [6]:
from openai import OpenAI
import json
import time
import os
import random

In [10]:
API_KEY = os.getenv("OPENAI_API_KEY")
print(f"API_KEY: ...{API_KEY[-10:]}") 

API_KEY: ...qsZHREcOQA


In [4]:
list_of_activities = ["Hiking", "Trail running", "Road cycling", "Mountain biking", "Canoeing", "Kayaking", "Stand-up paddleboarding", "Sailing", "Windsurfing", "Kitesurfing", "Open-water swimming", "Beach day", "Picnic", "Outdoor yoga", "Rock climbing", "Bouldering", "Camping", "Backpacking", "Birdwatching", "Fishing", "Hunting", "Photography walk", "City walking tour", "Outdoor concert/festival/parade", "Farmers’ market", "Stargazing", "Drone flying", "Golf", "Tennis", "Pickleball", "Soccer", "Basketball", "Baseball/softball", "Ultimate frisbee", "Disc golf", "Horseback riding", "ATV/Off-road", "Snowshoeing", "Cross-country skiing", "Alpine skiing", "Snowboarding", "Ice skating", "Sledding", "Winter hiking"]

In [3]:
client = OpenAI(api_key=API_KEY)

def get_recommendation(row):
    activity_list = row["preferred_activities"]
    weather_forecast = row["weather"]

    system_prompt = """
You are an outdoor-activity suitability assistant. 
You DO NOT forecast — you interpret the provided weather summary.
Be concise, practical, and safety-aware.
Always return a compact JSON object as described below.
Never add text outside the JSON.

Default heuristics (used only if no custom thresholds provided):
- Parade/Picnic: avoid precipitation ≥2 mm/day; wind <9 m/s; comfort ~0–28 °C.
- Hiking/Walking: avoid storms or heavy rain ≥5 mm/day; ok temp −5…30 °C; wind <12 m/s.
- Camping: avoid heavy rain ≥6 mm/day; wind <10 m/s; temp 0…28 °C.
- Canoe/Kayak/SUP: prefer wind ≤8 m/s; avoid thunderstorms; temp 10…28 °C; rain <5 mm/day.
- Sailing/Windsurf/Kitesurf: good wind 5–14 m/s; caution with gusts >14 m/s or storms.
- Cycling: avoid heavy rain ≥5 mm/day and gusts >14 m/s; temp −2…32 °C.
- Running: avoid thunderstorms; temp −5…30 °C; humidex/heat index >32 °C ⇒ caution.
- Alpine/Cross-country Skiing/Snowshoeing: prefer snow depth ≥10 cm or snow cover ≥50%; ok temp ≤2 °C; wind chill ≤ −20 °C ⇒ caution/no-go.
- Photography/Stargazing: prefer cloud cover <40%; wind <10 m/s; no precipitation.
- Fishing: light wind <8 m/s; avoid thunderstorms; light rain is tolerable.

RATING rules:
- GO: all conditions within comfort thresholds
- CAUTION: minor or borderline issues
- NO-GO: significant safety/weather limitations
"""

    user_prompt = f'''
Evaluate the suitability of each preferred activity given the weather data.

Input JSON:
{{
  "preferred_activities": {json.dumps(activity_list, ensure_ascii=False)},
  "weather": {json.dumps(weather_forecast, ensure_ascii=False)}
}}

Output STRICTLY as JSON with these keys:
{{
  "rating": "GO|CAUTION|NO-GO", // only one rating for all activities
  "why": ["<=12 words", "<=12 words"],  // one for each activity
  "alternatives": ["<=3 words", "<=3 words"],
  "one_liner": "<=22 words summary in English"
}}
'''

    for attempt in range(3):
        try:
            response = client.chat.completions.create(
                model="gpt-5-mini",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt},
                ],
            )

            answer = response.choices[0].message.content.strip()

            # attempt to extract JSON
            json_start = answer.find("{")
            json_end = answer.rfind("}") + 1
            answer_json = answer[json_start:json_end]
            result = json.loads(answer_json)
            return result

        except Exception as e:
            print(f"Error (attempt {attempt+1}): {e}")
            time.sleep(4 * (attempt + 1))
    return None


In [7]:
row = {
    "preferred_activities": random.sample(list_of_activities, 3),
    "weather": {
        "temp_c": {"min": 11, "max": 19},
        "wind_ms": {"mean": 8.5, "gust": 12.0},
        "precip_mm_day": 1.5,
        "thunderstorm_risk": "low",
        "cloud_cover_pct": 40,
        "snow_depth_cm": 0
    }
}

print(row['preferred_activities'])
result = get_recommendation(row)
print(json.dumps(result, indent=2, ensure_ascii=False))


['Sailing', 'Outdoor concert/festival/parade', 'Baseball/softball']
{
  "rating": "CAUTION",
  "why": [
    "Wind 8.5 m/s with 12 m/s gusts — favorable for sailing",
    "Wind near comfort limit; light rain may affect tents/stage",
    "Light rain, moderate wind — play generally unaffected"
  ],
  "alternatives": [
    "Photography",
    "Picnic"
  ],
  "one_liner": "Moderate wind and light rain — sailing and baseball OK; outdoor concert may need wind/tent precautions."
}
